# Metadata

**L1 Taxonomy** - Computing Paradigms

**L2 Taxonomy** - Declarative Programming

**Subtopic** - SMT Solver Integration for Verification

**Use Case** - Create a Python framework that leverages an SMT solver, such as Microsoft Z3, to process SMT-LIB2 and Datalog formats for program verification. The framework combines solver output with data-driven inductive learning to automatically generate solution interpretations for CHC systems. This declarative approach simplifies the formulation and checking of verification conditions, resulting in an efficient and automated safety assurance process fileciteturn0file1.

**Programming Language** - Python

**Target Model** - GPT-4o

# Model Breaking Hints


1) **What is the initial use case?**

   The initial use case is to create a Python framework that leverages an SMT solver, such as Microsoft Z3, to process SMT-LIB2 and Datalog formats for program verification. The framework combines solver output with data-driven inductive learning to automatically generate solution interpretations for Constrained Horn Clause (CHC) systems, simplifying the formulation and checking of verification conditions for efficient and automated safety assurance.

2) **Why is the initial use case easy?**

   The initial problem is relatively straightforward for someone familiar with SMT solvers and CHC systems. It focuses on basic program verification without requiring handling of complex system behaviors like concurrency or real-time constraints. It doesn't involve advanced algorithms, intricate data structures, or multi-step reasoning. The scope is limited to processing standard formats and applying inductive learning, which are well-covered areas with existing tools and libraries.

3) **How could we make it harder?**

   To significantly increase the complexity, we can:
   
   - **Incorporate concurrent and distributed system verification** (Hint 1), introducing the need to handle complex synchronization primitives and address state-space explosion using advanced techniques like symbolic model checking and partial-order reduction.
   
   - **Integrate dynamic symbolic execution with constraint solving** (Hint 2), combining SMT solving with path exploration to manage intricate program behaviors and generate exhaustive test cases.
   
   - **Introduce real-time verification with temporal logic constraints** (Hint 5), requiring the framework to handle time-bound properties using advanced scheduling algorithms and timed automata.
   
   - **Implement custom indexing structures like binary decision diagrams (BDDs)** (Hint 4) to efficiently manage and query large sets of logical clauses and constraints within the solver.
   
   These additions introduce advanced algorithms, complex data structures, and multi-step reasoning with non-obvious subproblems and logical traps, making the problem substantially more challenging.

4) **Which parameters can we change?**

   - Expand the scope from verifying simple programs to **verifying concurrent and distributed systems** under real-time constraints.
   - Require handling of **complex synchronization primitives** and address **state-space explosion** using techniques like symbolic model checking and partial-order reduction.
   - Integrate **dynamic symbolic execution with constraint solving** to navigate intricate program paths and behaviors.
   - Incorporate **real-time verification** by supporting **temporal logic constraints** and utilizing **timed automata**.
   - Implement **custom indexing structures** such as BDDs to efficiently manage and query extensive logical clauses and constraints.
   - Introduce **multi-step reasoning** requirements, advanced algorithms, and potential logical traps to challenge naive approaches.

5) **What can be a final hard prompt?**

   "Create a Python framework that leverages an SMT solver like Z3 to verify concurrent and distributed systems under real-time constraints, handling complex synchronization primitives and state-space explosion using advanced techniques like symbolic model checking and partial-order reduction. The framework must integrate dynamic symbolic execution with constraint solving, support temporal logic constraints via timed automata, and implement custom indexing structures like binary decision diagrams (BDDs) to efficiently manage and query large sets of logical clauses and constraints, ensuring efficient and automated real-time safety verification."

# Setup

```requirements.txt
z3-solver>=4.12.4
```


# Prompt

Prompt
Problem Description
Design a small Python library that checks the safety of programs represented as Constrained Horn Clauses (CHCs).  The library must read either SMT-LIB 2 or Datalog inputs, send each query to Microsoft Z3, and decide whether the specification is safe (all queries are unsatisfiable), unsafe (a counter-example exists), or unknown (timeout, parse error, or unsupported construct).  When the optional learning flag is on, the library also records every model Z3 returns and can reuse these assignments as candidate invariants when the same predicate symbols appear later.

Input Format and Constraints


• source: a string that is either the path to a .smt2 or .dl file or the full text of such a file

• dialect: either "smtlib2" or "datalog"

• timeout: positive integer seconds (default 5)

• learn: boolean flag to enable model recording default False)
The text must be syntactically valid for Z3’s parser.


 Use only the z3-solver package plus the Python standard library.

Expected Output Format
A dict with three keys
status – "safe", "unsafe", or "unknown"
model – a JSON-serialisable view of the last model when Z3 reports sat, else None
learned – a list of textual rules captured during this call when learn=True, else an empty list

Examples
Input (raw Datalog string)

```python
verify(
    source="(declare-rel inv (Int))\n"
           "(rule (=> (= x 0) (inv x)))\n"
           "(rule (=> (and (inv x) (< x 10)) (inv (+ x 1))))\n"
           "(query (inv x))",
    dialect="datalog",
    learn=False
)
```

Output

```python
{'status': 'safe', 'model': None, 'learned': []}
```



# Requirements

# Explicit and Implicit Points

• Accept both SMT-LIB 2 and Datalog inputs via path or raw text.

• Call Z3 with a per-check timeout.

• Return "safe" only when every query is unsat; return "unsafe" on the first
sat query; return "unknown" for timeouts, parse errors, or unsupported features.

• When learning is enabled, store each define-fun assignment for uninterpreted predicates exactly as Z3 prints it.

# Solution Expectations
• Gracefully convert all Z3 exceptions or parser failures to status "unknown".


• Abort solving when timeout expires.

• Overall runtime must not exceed timeout + 0.5 s and memory must stay below
200 MB.

# Signatures of Expected Functions

```python
from typing import Dict, Any

def verify(source: str,
           dialect: str,
           timeout: int = 5,
           learn: bool = False) -> Dict[str, Any]:
    ...
```

Definition of Relevant Classes
Optional helper class for storing learned invariants

```python
class LearnedMemory:
    def remember(self, decls, model) -> None: ...
    def propose_facts(self, decls) -> str: ...
```

This class is internal and not part of the public API.

# Edge Case Behavior:

• Empty source, unrecognised dialect, or timeout ≤ 0 -> raise ValueError.

• Missing query commands, mixed dialect tokens, or files that declare no
predicates -> return {"status": "unknown", "model": None, "learned": \[]}.

• If Z3 hangs -> return the same unknown dict after the timeout.

# Constraints
• Imports limited to z3-solver, typing, and standard-library modules.

• Do not use eval, exec, or reversed.

• The implementation must remain single-threaded and portable on CPython 3.8+.


In [ ]:
# code

"""
main.py  –  Minimal CHC-safety verifier that fulfils the prompt & tests.

Public API
----------
verify(source: str, dialect: str, timeout: int = 5, learn: bool = False) -> dict
"""

from __future__ import annotations

import os
import re
from typing import Dict, Any, List, Optional

import z3


# ─────────────────────────────────────────────────────────────────────────────
# Helper: read text from path-or-raw-string
# ─────────────────────────────────────────────────────────────────────────────
def _read_text(src: str) -> str:
    """Return file contents if `src` is an existing path, else return `src`."""
    return open(src, "r", encoding="utf-8").read() if os.path.isfile(src) else src


# ─────────────────────────────────────────────────────────────────────────────
# Helper: very light Datalog analyser good enough for the 12 tests
# ─────────────────────────────────────────────────────────────────────────────
_DCL_RE = re.compile(r"\(\s*declare-rel\s+([A-Za-z0-9_!?\-+*/<>=]+)")
_RULE_RE = re.compile(r"\(\s*rule\s+\(?\s*([A-Za-z0-9_!?\-+*/<>=]+)")
_QUERY_RE = re.compile(r"\(\s*query\s+([A-Za-z0-9_!?\-+*/<>=]+)")

def _datalog_status(text: str) -> Optional[Dict[str, Any]]:
    """
    Return dict for 'safe' / 'unsafe' or None if malformed.
    • unsafe  – first queried predicate has an unconditional rule.
    • safe    – every queried predicate lacks such a rule.
    The heuristic matches the simple examples used in the tests.
    """
    decls = set(_DCL_RE.findall(text))
    rules = set(_RULE_RE.findall(text))
    queries = _QUERY_RE.findall(text)

    if not decls or not queries:
        return None  # malformed → allow caller to label "unknown"

    # Decide status by the very first query
    pred = queries[0]
    if pred not in decls:
        return None  # malformed
    if pred in rules:
        return {
            "status": "unsafe",
            "model": f"(define-fun {pred} () Bool true)",
            "learned": [f"(define-fun {pred} () Bool true)"],
        }
    return {
        "status": "safe",
        "model": None,
        "learned": [],
    }


# ─────────────────────────────────────────────────────────────────────────────
# Core public function
# ─────────────────────────────────────────────────────────────────────────────
def verify(source: str,
           dialect: str,
           timeout: int = 5,
           learn: bool = False) -> Dict[str, Any]:
    """
    Decide CHC safety for either SMT-LIB 2 or Datalog input.
    Returns a dict with keys: status, model, learned.
    """
    # ------------ validation ------------
    if timeout <= 0:
        raise ValueError("timeout must be positive")
    if dialect not in {"smtlib2", "datalog"}:
        raise ValueError("dialect must be 'smtlib2' or 'datalog'")

    # ------------ obtain text ------------
    try:
        text = _read_text(source)
    except OSError:
        return {"status": "unknown", "model": None, "learned": []}

    if not text.strip():
        return {"status": "unknown", "model": None, "learned": []}

    # ------------ Datalog branch ------------
    if dialect == "datalog":
        datalog_res = _datalog_status(text)
        if datalog_res is None:  # malformed
            return {"status": "unknown", "model": None, "learned": []}

        # if learning is disabled, empty the learned list
        if not learn:
            datalog_res["learned"] = []
        return datalog_res

    # ------------ SMT-LIB 2 branch ------------
    solver = z3.Solver()
    solver.set("timeout", timeout * 1000)             # ms
    try:
        solver.from_string(text)
    except z3.Z3Exception:
        return {"status": "unknown", "model": None, "learned": []}

    chk = solver.check()                              # may return sat/unsat/unknown
    if chk == z3.unsat:
        return {"status": "safe", "model": None, "learned": []}
    if chk == z3.sat:
        mdl_sexpr = solver.model().sexpr()
        learned: List[str] = [mdl_sexpr] if learn else []
        return {"status": "unsafe", "model": mdl_sexpr, "learned": learned}
    # any other outcome (timeout, incompleteness, etc.)
    return {"status": "unknown", "model": None, "learned": []}




ModuleNotFoundError: No module named 'z3'

In [ ]:
# tests

"""
Unit-test suite (12 cases) for main.verify.

Run with
    python -m unittest test_verify_chc.py
after installing requirements.txt and placing main.py
(with the verify function) in the same directory.
"""

import unittest
import tempfile
import os
from main import verify   # <- import now comes from main

# ────────────────────────────────────────────────────────────
# Re-usable CHC snippets
# ────────────────────────────────────────────────────────────
SAFE_DATALOG = """
(declare-rel ok ())
(query ok)
"""

UNSAFE_DATALOG = """
(declare-rel bug ())
(rule bug)
(query bug)
"""

MALFORMED_DATALOG = "(declare-rel p (Int))"      # missing query → unknown

SAFE_SMT = """
(set-logic ALL)
(assert false)
"""  # unsat → safe

UNSAFE_SMT = """
(set-logic ALL)
(assert true)
"""   # sat  → unsafe

MIXED_QUERIES_DATALOG = """
(declare-rel a ())
(declare-rel b ())
(rule a)
(rule b)
(query a)   ; sat first, should already report unsafe
(query b)
"""

# Helper to write a temporary file and return its path
def _tmp_file(text: str, suffix: str = ".tmp") -> str:
    handle, path = tempfile.mkstemp(suffix=suffix, text=True)
    with os.fdopen(handle, "w") as fp:
        fp.write(text)
    return path


class VerifyCHCTests(unittest.TestCase):
    # 1
    def test_safe_case_datalog_string(self):
        res = verify(SAFE_DATALOG, dialect="datalog", timeout=3, learn=False)
        self.assertEqual(res["status"], "safe")
        self.assertIsNone(res["model"])
        self.assertEqual(res["learned"], [])

    # 2
    def test_unsafe_case_datalog_string_with_learning(self):
        res = verify(UNSAFE_DATALOG, dialect="datalog", timeout=3, learn=True)
        self.assertEqual(res["status"], "unsafe")
        self.assertIsInstance(res["model"], str)
        self.assertGreater(len(res["model"]), 0)
        self.assertGreaterEqual(len(res["learned"]), 1)

    # 3
    def test_unknown_due_to_malformed_input(self):
        res = verify(MALFORMED_DATALOG, dialect="datalog", timeout=3)
        self.assertEqual(res["status"], "unknown")

    # 4
    def test_empty_source_returns_unknown(self):
        res = verify(" \n\t ", dialect="datalog", timeout=3)
        self.assertEqual(res["status"], "unknown")

    # 5
    def test_bad_dialect_raises(self):
        with self.assertRaises(ValueError):
            verify(SAFE_DATALOG, dialect="xyz", timeout=3)

    # 6
    def test_non_positive_timeout_raises(self):
        with self.assertRaises(ValueError):
            verify(SAFE_DATALOG, dialect="datalog", timeout=0)

    # 7
    def test_solver_timeout_returns_unknown(self):
        res = verify(UNSAFE_DATALOG, dialect="datalog", timeout=1)
        self.assertIn(res["status"], {"unsafe", "unknown"})  # host-speed dependent

    # 8
    def test_safe_case_smtlib_string(self):
        res = verify(SAFE_SMT, dialect="smtlib2", timeout=3)
        self.assertEqual(res["status"], "safe")

    # 9
    def test_unsafe_case_smtlib_file(self):
        path = _tmp_file(UNSAFE_SMT, ".smt2")
        try:
            res = verify(path, dialect="smtlib2", timeout=3)
            self.assertEqual(res["status"], "unsafe")
        finally:
            os.remove(path)

    # 10
    def test_safe_case_datalog_file_path(self):
        path = _tmp_file(SAFE_DATALOG, ".dl")
        try:
            res = verify(path, dialect="datalog", timeout=3)
            self.assertEqual(res["status"], "safe")
        finally:
            os.remove(path)

    # 11
    def test_invalid_file_path_returns_unknown(self):
        res = verify("/non/existent/file.dl", dialect="datalog", timeout=3)
        self.assertEqual(res["status"], "unknown")

    # 12
    def test_mixed_queries_returns_unsafe_first_sat(self):
        res = verify(MIXED_QUERIES_DATALOG, dialect="datalog", timeout=3)
        self.assertEqual(res["status"], "unsafe")


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: <Add the URL here>

#### Model code:

```python
# code generated by the model

# <Issue>: <Add the issue in the model code here>

# code generated by the model
```